In [91]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from scipy.optimize import curve_fit

In [92]:
# Indlæs data

loc1 = r"C:\Users\marcu\OneDrive\Documents\GitHub\Solpletter_projekt\Magnus carlsen\data\13-month smoothed monthly total sunspot number 1-1749 to now.csv"
loc2 = r"C:\Users\marcu\OneDrive\Documents\GitHub\Solpletter_projekt\Magnus carlsen\data\WSO_smoothed.csv"

column_names = ['Year', 'Year_fraction', 'Monthly_smoothed_sunspot_number', 'Uncertainty', 'N_obs', 'DefProvMarker']
sunspot_raw_data = pd.read_csv(loc1, names=column_names, delimiter=';', skiprows=1)
sunspot_raw_data = sunspot_raw_data.iloc[6:-6]

# Sunspot data
SSN_date_fraction_arr = sunspot_raw_data['Year_fraction'].values.astype(float)
SSN_arr = sunspot_raw_data['Monthly_smoothed_sunspot_number'].values.astype(float)
SSN_d_arr = sunspot_raw_data['Uncertainty'].values.astype(float)

# WSO data
wso_raw_data = np.genfromtxt(loc2, delimiter=',', skip_header=1)

wso_date_fraction_arr = wso_raw_data[:, 0]
wso_number_arr = wso_raw_data[:, 1]

# Størrelse på punkterne i scatterplottet
ScatterSize = 1




# Fits med n sinusfkt.

In [134]:
import numpy as np
from scipy.optimize import curve_fit

# Enkelt sinfkt
def single_sine(x, amplitude, frequency, phase, offset):
    return amplitude * np.sin(frequency * x + phase) + offset

# n sinfkt med egne parametre
def multiple_sines(x, *params):
    n = len(params) // 4
    result = np.zeros_like(x)
    for i in range(n):
        result += single_sine(x, *params[i*4:(i+1)*4])
    return result

params = []
n_waves = 5

X = SSN_date_fraction_arr
Y = SSN_arr

for i in range(n_waves):
    if i == 0:
        # Fit enkelt sinfkt
        popt, _ = curve_fit(single_sine, X, Y)
    else:
        # Fit flere sinfkt
        popt, _ = curve_fit(multiple_sines, X, Y, sigma=SSN_d_arr, absolute_sigma=True, p0=params)
    params.extend(popt[:4])

# Plot data og fit
plt.plot(X, Y, label='Data')
plt.plot(X, multiple_sines(X, *params), label='Fit')
plt.legend()
plt.show()




RuntimeError: Optimal parameters not found: Number of calls to function has reached maxfev = 3400.